In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
from sklearn.preprocessing import MultiLabelBinarizer

In [45]:
df_claims = pd.read_csv('../0.Data/Claims.csv')

In [46]:
df_claims.dtypes

MemberID                   int64
ProviderID               float64
Vendor                   float64
PCP                      float64
Year                      object
Specialty                 object
PlaceSvc                  object
PayDelay                  object
LengthOfStay              object
DSFS                      object
PrimaryConditionGroup     object
CharlsonIndex             object
ProcedureGroup            object
SupLOS                     int64
dtype: object

In [47]:
df_claims.head()

,MemberID,ProviderID,Vendor,PCP,Year,Specialty,PlaceSvc,PayDelay,LengthOfStay,DSFS,PrimaryConditionGroup,CharlsonIndex,ProcedureGroup,SupLOS
0,42286978,8013252.0,172193.0,37796.0,Y1,Surgery,Office,28,NaN,8- 9 months,NEUMENT,0,MED,0
1,97903248,3316066.0,726296.0,5300.0,Y3,Internal,Office,50,NaN,7- 8 months,NEUMENT,1-2,EM,0
2,2759427,2997752.0,140343.0,91972.0,Y3,Internal,Office,14,NaN,0- 1 month,METAB3,0,EM,0
3,73570559,7053364.0,240043.0,70119.0,Y3,Laboratory,Independent Lab,24,NaN,5- 6 months,METAB3,1-2,SCS,0
4,11837054,7557061.0,496247.0,68968.0,Y2,Surgery,Outpatient Hospital,27,NaN,4- 5 months,FXDISLC,1-2,EM,0


In [51]:
df_claims.loc[(df_claims['MemberID']==33120) & (df_claims['Year']=='Y2')].shape

(43, 14)

In [5]:
df_claims.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2668990 entries, 0 to 2668989
Data columns (total 14 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   MemberID               int64  
 1   ProviderID             float64
 2   Vendor                 float64
 3   PCP                    float64
 4   Year                   object 
 5   Specialty              object 
 6   PlaceSvc               object 
 7   PayDelay               object 
 8   LengthOfStay           object 
 9   DSFS                   object 
 10  PrimaryConditionGroup  object 
 11  CharlsonIndex          object 
 12  ProcedureGroup         object 
 13  SupLOS                 int64  
dtypes: float64(3), int64(2), object(9)
memory usage: 285.1+ MB


In [6]:
#Count missing data
def count_missing(df):   
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    print(missing_data.head(20))
    return missing_data

count_missing(df_claims)

                         Total   Percent
LengthOfStay           2597392  0.973174
DSFS                     52770  0.019772
Vendor                   24856  0.009313
ProviderID               16264  0.006094
PrimaryConditionGroup    11410  0.004275
Specialty                 8405  0.003149
PlaceSvc                  7632  0.002860
PCP                       7492  0.002807
ProcedureGroup            3675  0.001377
SupLOS                       0  0.000000
CharlsonIndex                0  0.000000
PayDelay                     0  0.000000
Year                         0  0.000000
MemberID                     0  0.000000


,Total,Percent
LengthOfStay,2597392,0.973174
DSFS,52770,0.019772
Vendor,24856,0.009313
ProviderID,16264,0.006094
PrimaryConditionGroup,11410,0.004275
Specialty,8405,0.003149
PlaceSvc,7632,0.002860
PCP,7492,0.002807
ProcedureGroup,3675,0.001377
SupLOS,0,0.000000


In [7]:
#Update paydelay
df_claims.PayDelay.replace({'162+': '162'}, inplace=True)
df_claims['PayDelay'] = df_claims['PayDelay'].astype(int)

In [8]:
df_claims.dtypes

MemberID                   int64
ProviderID               float64
Vendor                   float64
PCP                      float64
Year                      object
Specialty                 object
PlaceSvc                  object
PayDelay                   int32
LengthOfStay              object
DSFS                      object
PrimaryConditionGroup     object
CharlsonIndex             object
ProcedureGroup            object
SupLOS                     int64
dtype: object

In [9]:
#Update DSFS
df_claims.loc[df_claims['DSFS']=='0- 1 month','DSFS'] = 1
df_claims.loc[df_claims['DSFS']=='1- 2 months','DSFS'] = 2
df_claims.loc[df_claims['DSFS']=='2- 3 months','DSFS'] = 3
df_claims.loc[df_claims['DSFS']=='3- 4 months','DSFS'] = 4
df_claims.loc[df_claims['DSFS']=='4- 5 months','DSFS'] = 5
df_claims.loc[df_claims['DSFS']=='5- 6 months','DSFS'] = 6
df_claims.loc[df_claims['DSFS']=='6- 7 months','DSFS'] = 7
df_claims.loc[df_claims['DSFS']=='7- 8 months','DSFS'] = 8
df_claims.loc[df_claims['DSFS']=='8- 9 months','DSFS'] = 9
df_claims.loc[df_claims['DSFS']=='9-10 months','DSFS'] = 10
df_claims.loc[df_claims['DSFS']=='10-11 months','DSFS'] = 11
df_claims.loc[df_claims['DSFS']=='11-12 months','DSFS'] = 12
df_claims['DSFS'].fillna(0, inplace=True)
df_claims['DSFS'] = df_claims['DSFS'].astype(int)

In [10]:
df_claims.dtypes

MemberID                   int64
ProviderID               float64
Vendor                   float64
PCP                      float64
Year                      object
Specialty                 object
PlaceSvc                  object
PayDelay                   int32
LengthOfStay              object
DSFS                       int32
PrimaryConditionGroup     object
CharlsonIndex             object
ProcedureGroup            object
SupLOS                     int64
dtype: object

In [11]:
#Update Charlson
df_claims.loc[df_claims['CharlsonIndex']=='0','CharlsonIndex'] = 0
df_claims.loc[df_claims['CharlsonIndex']=='1-2','CharlsonIndex'] = 2
df_claims.loc[df_claims['CharlsonIndex']=='3-4','CharlsonIndex'] = 4
df_claims.loc[df_claims['CharlsonIndex']=='5+','CharlsonIndex'] = 6
df_claims['CharlsonIndex'] = df_claims['CharlsonIndex'].astype(int)

In [12]:
df_claims.dtypes

MemberID                   int64
ProviderID               float64
Vendor                   float64
PCP                      float64
Year                      object
Specialty                 object
PlaceSvc                  object
PayDelay                   int32
LengthOfStay              object
DSFS                       int32
PrimaryConditionGroup     object
CharlsonIndex              int32
ProcedureGroup            object
SupLOS                     int64
dtype: object

In [13]:
df_claims['MemberID'] = df_claims['MemberID'].astype(object)
df_claims['ProviderID'] = df_claims['ProviderID'].astype(object)
#convert Vendor từ float64 thành object
df_claims['Vendor'] = df_claims['Vendor'].astype(object)
#convert PCP từ float64 thành object
df_claims['PCP'] = df_claims['PCP'].astype(object)

In [14]:
df_claims.dtypes

MemberID                 object
ProviderID               object
Vendor                   object
PCP                      object
Year                     object
Specialty                object
PlaceSvc                 object
PayDelay                  int32
LengthOfStay             object
DSFS                      int32
PrimaryConditionGroup    object
CharlsonIndex             int32
ProcedureGroup           object
SupLOS                    int64
dtype: object

In [15]:
def impute_nan_create_category(DataFrame,ColName):
     DataFrame[ColName] = np.where(DataFrame[ColName].isnull(),"Unknown",DataFrame[ColName])

In [17]:
#impute ProviderID
impute_nan_create_category(df_claims,'ProviderID')

#impute Vendor
impute_nan_create_category(df_claims,'Vendor')

#impute PCP
impute_nan_create_category(df_claims,'PCP')

#impute Specialty
impute_nan_create_category(df_claims,'Specialty')

#impute PlaceSvc
impute_nan_create_category(df_claims,'PlaceSvc')

#impute PrimaryConditionGroup 
impute_nan_create_category(df_claims,'PrimaryConditionGroup')

#impute ProcedureGroup 
impute_nan_create_category(df_claims,'ProcedureGroup')

In [18]:
count_missing(df_claims)

                         Total   Percent
LengthOfStay           2597392  0.973174
SupLOS                       0  0.000000
ProcedureGroup               0  0.000000
CharlsonIndex                0  0.000000
PrimaryConditionGroup        0  0.000000
DSFS                         0  0.000000
PayDelay                     0  0.000000
PlaceSvc                     0  0.000000
Specialty                    0  0.000000
Year                         0  0.000000
PCP                          0  0.000000
Vendor                       0  0.000000
ProviderID                   0  0.000000
MemberID                     0  0.000000


,Total,Percent
LengthOfStay,2597392,0.973174
SupLOS,0,0.000000
ProcedureGroup,0,0.000000
CharlsonIndex,0,0.000000
PrimaryConditionGroup,0,0.000000
DSFS,0,0.000000
PayDelay,0,0.000000
PlaceSvc,0,0.000000
Specialty,0,0.000000
Year,0,0.000000


In [19]:
#Update LengthOfStay
df_claims['LOS_UNKNOWN'] = 0
df_claims['LOS_SUPRESSED'] = 0
df_claims['LOS_KNOWN'] = 0
m = (df_claims['LengthOfStay'].isna()) & (df_claims['SupLOS'] == 0)
df_claims.loc[m, 'LOS_UNKNOWN'] = 1
m = (df_claims['LengthOfStay'].isna()) & (df_claims['SupLOS'] == 1)
df_claims.loc[m, 'LOS_SUPRESSED'] = 1
m = (df_claims['LengthOfStay'].notna()) & (df_claims['SupLOS'] == 1)
df_claims.loc[m, 'LOS_KNOWN'] = 1

df_claims.loc[df_claims['LengthOfStay']=='1 day','LengthOfStay'] = 1
df_claims.loc[df_claims['LengthOfStay']=='2 days','LengthOfStay'] = 2
df_claims.loc[df_claims['LengthOfStay']=='3 days','LengthOfStay'] = 3
df_claims.loc[df_claims['LengthOfStay']=='4 days','LengthOfStay'] = 4
df_claims.loc[df_claims['LengthOfStay']=='5 days','LengthOfStay'] = 5
df_claims.loc[df_claims['LengthOfStay']=='6 days','LengthOfStay'] = 6
df_claims.loc[df_claims['LengthOfStay']=='1- 2 weeks','LengthOfStay'] = 11
df_claims.loc[df_claims['LengthOfStay']=='2- 4 weeks','LengthOfStay'] = 21
df_claims.loc[df_claims['LengthOfStay']=='4- 8 weeks','LengthOfStay'] = 42
df_claims.loc[df_claims['LengthOfStay']=='26+ weeks','LengthOfStay'] = 180
df_claims['LengthOfStay'].fillna(0, inplace=True)
df_claims['LengthOfStay'] = df_claims['LengthOfStay'].astype(int)

In [20]:
df_claims.dtypes

MemberID                 object
ProviderID               object
Vendor                   object
PCP                      object
Year                     object
Specialty                object
PlaceSvc                 object
PayDelay                  int32
LengthOfStay              int32
DSFS                      int32
PrimaryConditionGroup    object
CharlsonIndex             int32
ProcedureGroup           object
SupLOS                    int64
LOS_UNKNOWN               int64
LOS_SUPRESSED             int64
LOS_KNOWN                 int64
dtype: object

In [21]:
df_claims['PrimaryConditionGroup_Specialty'] = df_claims['PrimaryConditionGroup'] + '_' + df_claims['Specialty']
df_claims['ProcedureGroup_Specialty'] = df_claims['ProcedureGroup'] + '_' + df_claims['Specialty']
df_claims['ProcedureGroup_PrimaryConditionGroup'] = df_claims['ProcedureGroup'] + '_' + df_claims['PrimaryConditionGroup']
df_claims['PrimaryConditionGroup_PlaceSvc'] = df_claims['PrimaryConditionGroup'] + '_' + df_claims['PlaceSvc']

In [22]:
count_missing(df_claims)

                                      Total  Percent
PrimaryConditionGroup_PlaceSvc            0      0.0
DSFS                                      0      0.0
ProviderID                                0      0.0
Vendor                                    0      0.0
PCP                                       0      0.0
Year                                      0      0.0
Specialty                                 0      0.0
PlaceSvc                                  0      0.0
PayDelay                                  0      0.0
LengthOfStay                              0      0.0
PrimaryConditionGroup                     0      0.0
ProcedureGroup_PrimaryConditionGroup      0      0.0
CharlsonIndex                             0      0.0
ProcedureGroup                            0      0.0
SupLOS                                    0      0.0
LOS_UNKNOWN                               0      0.0
LOS_SUPRESSED                             0      0.0
LOS_KNOWN                                 0   

,Total,Percent
PrimaryConditionGroup_PlaceSvc,0,0.0
DSFS,0,0.0
ProviderID,0,0.0
Vendor,0,0.0
PCP,0,0.0
Year,0,0.0
Specialty,0,0.0
PlaceSvc,0,0.0
PayDelay,0,0.0
LengthOfStay,0,0.0


In [23]:
#create a summary table at the member/year level
df_new = pd.DataFrame()
df_group = df_claims.groupby(['Year','MemberID'])
df_new = df_group.agg({
    'ProviderID': ['count','nunique'],
    'Vendor': 'nunique',
    'PCP': 'nunique',
    'PlaceSvc': 'nunique',
    'Specialty': 'nunique',
    'PrimaryConditionGroup': 'nunique',
    'ProcedureGroup': 'nunique',
    'PayDelay': ['sum','max','min', 'mean','std'],
    'LengthOfStay': ['sum','max','min','mean', 'std'],
    'LOS_UNKNOWN': ['sum'],
    'LOS_SUPRESSED': ['sum'],
    'LOS_KNOWN': ['sum'],
    'DSFS': ['sum','max','min','mean', 'std'],
    'CharlsonIndex': ['sum','max','min','mean', 'std'],
    'PrimaryConditionGroup_Specialty': 'nunique',
    'ProcedureGroup_Specialty': 'nunique',
    'ProcedureGroup_PrimaryConditionGroup': 'nunique',
    'PrimaryConditionGroup_PlaceSvc': 'nunique',    
}).reset_index()
df_new.head()

Year MemberID ProviderID          Vendor     PCP PlaceSvc Specialty  \
                     count nunique nunique nunique  nunique   nunique   
0   Y1      210          8       4       4       2        3         3   
1   Y1     3197          5       3       3       1        2         2   
2   Y1     3889         13       7       7       1        3         4   
3   Y1     4187          4       3       3       1        1         3   
4   Y1     9063          4       2       2       1        1         2   

  PrimaryConditionGroup ProcedureGroup  ...      DSFS CharlsonIndex          \
                nunique        nunique  ...       std           sum max min   
0                     4              5  ...  3.338092             0   0   0   
1                     2              2  ...  5.272571             0   0   0   
2                     5              5  ...  0.277350             8   2   0   
3                     3              2  ...  3.593976             0   0   0   
4                     1              2  ...  0.500000             0   0   0   

                      PrimaryConditionGroup_Specialty  \
       mean       std                         nunique   
0  0.000000  0.000000                               4   
1  0.000000  0.000000                               3   
2  0.615385  0.960769                               5   
3  0.000000  0.000000                               4   
4  0.000000  0.000000                               2   

  ProcedureGroup_Specialty ProcedureGroup_PrimaryConditionGroup  \
                   nunique                              nunique   
0                        5                                    6   
1                        3                                    3   
2                        7                                    9   
3                        3                                    3   
4                        3                                    2   

  PrimaryConditionGroup_PlaceSvc  
                         nunique  
0                              4  
1                              3  
2                              5  
3                              3  
4                              1  

[5 rows x 37 columns]

In [24]:
df_group.head()

,MemberID,ProviderID,Vendor,PCP,Year,Specialty,PlaceSvc,PayDelay,LengthOfStay,DSFS,...,CharlsonIndex,ProcedureGroup,SupLOS,LOS_UNKNOWN,LOS_SUPRESSED,LOS_KNOWN,PrimaryConditionGroup_Specialty,ProcedureGroup_Specialty,ProcedureGroup_PrimaryConditionGroup,PrimaryConditionGroup_PlaceSvc
0,42286978,8.01325e+06,172193,37796,Y1,Surgery,Office,28,0,9,...,0,MED,0,1,0,0,NEUMENT_Surgery,MED_Surgery,MED_NEUMENT,NEUMENT_Office
1,97903248,3.31607e+06,726296,5300,Y3,Internal,Office,50,0,8,...,2,EM,0,1,0,0,NEUMENT_Internal,EM_Internal,EM_NEUMENT,NEUMENT_Office
2,2759427,2.99775e+06,140343,91972,Y3,Internal,Office,14,0,1,...,0,EM,0,1,0,0,METAB3_Internal,EM_Internal,EM_METAB3,METAB3_Office
3,73570559,7.05336e+06,240043,70119,Y3,Laboratory,Independent Lab,24,0,6,...,2,SCS,0,1,0,0,METAB3_Laboratory,SCS_Laboratory,SCS_METAB3,METAB3_Independent Lab
4,11837054,7.55706e+06,496247,68968,Y2,Surgery,Outpatient Hospital,27,0,5,...,2,EM,0,1,0,0,FXDISLC_Surgery,EM_Surgery,EM_FXDISLC,FXDISLC_Outpatient Hospital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2668944,9244526,7.05336e+06,240043,83829,Y1,Laboratory,Independent Lab,31,0,1,...,0,PL,0,1,0,0,RESPR4_Laboratory,PL_Laboratory,PL_RESPR4,RESPR4_Independent Lab
2668946,61057175,7.09060e+06,817122,48897,Y3,General Practice,Office,34,0,1,...,0,EM,0,1,0,0,MSC2a3_General Practice,EM_General Practice,EM_MSC2a3,MSC2a3_Office
2668954,79807678,9.97415e+06,781575,9524,Y2,Internal,Office,28,0,7,...,2,EM,0,1,0,0,ARTHSPIN_Internal,EM_Internal,EM_ARTHSPIN,ARTHSPIN_Office
2668961,46449277,822173,917318,27406,Y3,Internal,Office,20,0,2,...,0,EM,0,1,0,0,SKNAUT_Internal,EM_Internal,EM_SKNAUT,SKNAUT_Office


In [25]:
#Change columns name:
df_new.columns = ['Year', 'MemberID', 'no_Claims', 'no_Providers','no_Vendors','no_PCPs',
                    'no_PlaceSvcs', 'no_Specialities','no_PrimaryConditionGroups','no_ProcedureGroups',
                    'PayDelay_total','PayDelay_max','PayDelay_min','PayDelay_ave','PayDelay_stdev',
                    'LOS_sum','LOS_max','LOS_min','LOS_ave','LOS_std','LOS_TOT_UNKNOWN','LOS_TOT_SUPRESSED','LOS_TOT_KNOWN',
                    'DSFS_sum','DSFS_max','DSFS_min','DSFS_ave','DSFS_std',
                    'CharlsonIndex_sum','CharlsonIndex_max','CharlsonIndex_min','CharlsonIndex_ave','CharlsonIndex_std',
                    'no_PrimaryConditionGroup_Specialty','no_ProcedureGroup_Specialty','no_ProcedureGroup_PrimaryConditionGroup',
                    'no_PrimaryConditionGroup_PlaceSvc'
                 ]
df_new.head()

,Year,MemberID,no_Claims,no_Providers,no_Vendors,no_PCPs,no_PlaceSvcs,no_Specialities,no_PrimaryConditionGroups,no_ProcedureGroups,...,DSFS_std,CharlsonIndex_sum,CharlsonIndex_max,CharlsonIndex_min,CharlsonIndex_ave,CharlsonIndex_std,no_PrimaryConditionGroup_Specialty,no_ProcedureGroup_Specialty,no_ProcedureGroup_PrimaryConditionGroup,no_PrimaryConditionGroup_PlaceSvc
0,Y1,210,8,4,4,2,3,3,4,5,...,3.338092,0,0,0,0.000000,0.000000,4,5,6,4
1,Y1,3197,5,3,3,1,2,2,2,2,...,5.272571,0,0,0,0.000000,0.000000,3,3,3,3
2,Y1,3889,13,7,7,1,3,4,5,5,...,0.277350,8,2,0,0.615385,0.960769,5,7,9,5
3,Y1,4187,4,3,3,1,1,3,3,2,...,3.593976,0,0,0,0.000000,0.000000,4,3,3,3
4,Y1,9063,4,2,2,1,1,2,1,2,...,0.500000,0,0,0,0.000000,0.000000,2,3,2,1


In [26]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218415 entries, 0 to 218414
Data columns (total 37 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   Year                                     218415 non-null  object 
 1   MemberID                                 218415 non-null  int64  
 2   no_Claims                                218415 non-null  int64  
 3   no_Providers                             218415 non-null  int64  
 4   no_Vendors                               218415 non-null  int64  
 5   no_PCPs                                  218415 non-null  int64  
 6   no_PlaceSvcs                             218415 non-null  int64  
 7   no_Specialities                          218415 non-null  int64  
 8   no_PrimaryConditionGroups                218415 non-null  int64  
 9   no_ProcedureGroups                       218415 non-null  int64  
 10  PayDelay_total                  

In [27]:
count_missing(df_new)

                                   Total  Percent
PayDelay_stdev                     18443  0.08444
LOS_std                            18443  0.08444
CharlsonIndex_std                  18443  0.08444
DSFS_std                           18443  0.08444
no_PrimaryConditionGroup_PlaceSvc      0  0.00000
no_PrimaryConditionGroups              0  0.00000
PayDelay_ave                           0  0.00000
PayDelay_min                           0  0.00000
PayDelay_max                           0  0.00000
PayDelay_total                         0  0.00000
no_ProcedureGroups                     0  0.00000
no_Specialities                        0  0.00000
LOS_max                                0  0.00000
no_PlaceSvcs                           0  0.00000
no_PCPs                                0  0.00000
no_Vendors                             0  0.00000
no_Providers                           0  0.00000
no_Claims                              0  0.00000
MemberID                               0  0.00000


,Total,Percent
PayDelay_stdev,18443,0.08444
LOS_std,18443,0.08444
CharlsonIndex_std,18443,0.08444
DSFS_std,18443,0.08444
no_PrimaryConditionGroup_PlaceSvc,0,0.00000
no_PrimaryConditionGroups,0,0.00000
PayDelay_ave,0,0.00000
PayDelay_min,0,0.00000
PayDelay_max,0,0.00000
PayDelay_total,0,0.00000


In [28]:
df_new['DSFS_range'] = df_new['DSFS_max'] - df_new['DSFS_min']
df_new['CharlsonIndex_range'] = df_new['CharlsonIndex_max'] - df_new['CharlsonIndex_min']

In [29]:
df_new.loc[(df_new['MemberID']==69258001)]

,Year,MemberID,no_Claims,no_Providers,no_Vendors,no_PCPs,no_PlaceSvcs,no_Specialities,no_PrimaryConditionGroups,no_ProcedureGroups,...,CharlsonIndex_max,CharlsonIndex_min,CharlsonIndex_ave,CharlsonIndex_std,no_PrimaryConditionGroup_Specialty,no_ProcedureGroup_Specialty,no_ProcedureGroup_PrimaryConditionGroup,no_PrimaryConditionGroup_PlaceSvc,DSFS_range,CharlsonIndex_range
52589,Y1,69258001,5,3,3,1,2,3,3,4,...,0,0,0.000000,0.000000,5,4,5,4,2,0
125425,Y2,69258001,21,7,5,2,2,5,4,4,...,2,0,1.619048,0.804748,9,6,9,5,8,2
196545,Y3,69258001,24,8,7,1,2,5,8,5,...,2,0,1.916667,0.408248,13,8,13,10,11,2


In [30]:
df_new.shape

(218415, 39)

In [ ]:
"""
for value in df_claims['Specialty'].unique().tolist():  
    df_claims['Specialty_Count_'+value] = (df_claims['Specialty'] == value).astype(int)
    df_agg = df_groupby.agg({'Specialty_Count_'+ value:'sum'}).reset_index()
    df_new = pd.merge(df_new,df_agg, how = 'inner', on = ['MemberID','Year'])

"""

In [ ]:
'''
for value in df_claims['PlaceSvc'].unique().tolist():  
    df_claims['PlaceSvc_Count_'+value] = (df_claims['PlaceSvc'] == value).astype(int)
    df_agg = df_groupby.agg({'PlaceSvc_Count_'+ value:'sum'}).reset_index()
    df_new = pd.merge(df_new,df_agg, how = 'inner', on = ['MemberID','Year'])
'''

In [31]:
list_df_col = ['Specialty','PlaceSvc','PrimaryConditionGroup','ProcedureGroup']
for col in list_df_col:
    for value in df_claims[col].unique().tolist():
        df_claims[ str(col) + '_Count_' + str(value)] = (df_claims[col]==value).astype(int)
        df_agg = df_group.agg({str(col) + '_Count_' + str(value):'sum'}).reset_index()
        df_new = pd.merge(df_new, df_agg, how = 'inner', on = ['MemberID','Year'])

In [32]:
df_new.head()

,Year,MemberID,no_Claims,no_Providers,no_Vendors,no_PCPs,no_PlaceSvcs,no_Specialities,no_PrimaryConditionGroups,no_ProcedureGroups,...,ProcedureGroup_Count_SMS,ProcedureGroup_Count_SGS,ProcedureGroup_Count_SEOA,ProcedureGroup_Count_SNS,ProcedureGroup_Count_Unknown,ProcedureGroup_Count_SAS,ProcedureGroup_Count_SRS,ProcedureGroup_Count_SUS,ProcedureGroup_Count_SO,ProcedureGroup_Count_SMCD
0,Y1,210,8,4,4,2,3,3,4,5,...,0,0,0,0,0,0,0,0,0,0
1,Y1,3197,5,3,3,1,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
2,Y1,3889,13,7,7,1,3,4,5,5,...,0,0,0,0,0,0,0,0,0,0
3,Y1,4187,4,3,3,1,1,3,3,2,...,0,0,0,0,0,0,0,0,0,0
4,Y1,9063,4,2,2,1,1,2,1,2,...,0,0,0,0,0,0,0,0,0,0


In [35]:
for i in df_new.columns:
    print (i)

Year
MemberID
no_Claims
no_Providers
no_Vendors
no_PCPs
no_PlaceSvcs
no_Specialities
no_PrimaryConditionGroups
no_ProcedureGroups
PayDelay_total
PayDelay_max
PayDelay_min
PayDelay_ave
PayDelay_stdev
LOS_sum
LOS_max
LOS_min
LOS_ave
LOS_std
LOS_TOT_UNKNOWN
LOS_TOT_SUPRESSED
LOS_TOT_KNOWN
DSFS_sum
DSFS_max
DSFS_min
DSFS_ave
DSFS_std
CharlsonIndex_sum
CharlsonIndex_max
CharlsonIndex_min
CharlsonIndex_ave
CharlsonIndex_std
no_PrimaryConditionGroup_Specialty
no_ProcedureGroup_Specialty
no_ProcedureGroup_PrimaryConditionGroup
no_PrimaryConditionGroup_PlaceSvc
DSFS_range
CharlsonIndex_range
Specialty_Count_Surgery
Specialty_Count_Internal
Specialty_Count_Laboratory
Specialty_Count_Pediatrics
Specialty_Count_Rehabilitation
Specialty_Count_Diagnostic Imaging
Specialty_Count_Anesthesiology
Specialty_Count_Emergency
Specialty_Count_General Practice
Specialty_Count_Other
Specialty_Count_Obstetrics and Gynecology
Specialty_Count_Pathology
Specialty_Count_Unknown
PlaceSvc_Count_Office
PlaceSvc_Count_

In [39]:
df_new.loc[(df_new['MemberID']==18625637)][['Year','MemberID','Specialty_Count_Surgery']]

,Year,MemberID,Specialty_Count_Surgery
14252,Y1,18625637,4
89448,Y2,18625637,7
160856,Y3,18625637,12


In [ ]:
#df11=df.groupby('key1')['key2'].apply(lambda x: (x=='one').sum()).reset_index(name='count')

'''
df_new['pgc1'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='MSC2a3'))
df_new['pgc2'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='METAB3'))
df_new['pgc3'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='ARTHSPIN'))
df_new['pgc4'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='NEUMENT'))
df_new['pgc5'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='RESPR4'))
df_new['pgc6'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='MISCHRT'))
df_new['pgc7'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='SKNAUT'))
df_new['pgc8'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='GIBLEED'))
df_new['pgc9'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='INFEC4'))
df_new['pgc10'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='TRAUMA'))
'''

In [ ]:
'''
df_new['pgc11'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='HEART2'))
df_new['pgc12'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='RENAL3'))
df_new['pgc13'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='ROAMI'))
df_new['pgc14'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='MISCL5'))
df_new['pgc15'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='ODaBNCA'))
df_new['pgc16'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='UTI'))
df_new['pgc17'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='COPD'))
df_new['pgc18'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='GYNEC1'))
df_new['pgc19'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='CANCRB'))
df_new['pgc20'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='FXDISLC'))
'''

In [ ]:

'''
df_new['pgc21'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='AMI'))
df_new['pgc22'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='PRGNCY'))
df_new['pgc23'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='HEMTOL'))
df_new['pgc24'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='HEART4'))
df_new['pgc25'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='SEIZURE'))
df_new['pgc26'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='APPCHOL'))
df_new['pgc27'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='CHF'))
df_new['pgc28'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='GYNECA'))
df_new['pgc29'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x.isnull()))
df_new['pgc30'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='PNEUM'))
'''

In [ ]:
'''
df_new['pgc31'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='RENAL2'))
df_new['pgc32'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='GIOBSENT'))
df_new['pgc33'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='STROKE'))
df_new['pgc34'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='CANCRA'))
df_new['pgc35'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='FLaELEC'))
df_new['pgc36'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='MISCL1'))
df_new['pgc37'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='HIPFX'))
df_new['pgc38'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='METAB1'))
df_new['pgc39'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='PERVALV'))
df_new['pgc40'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='LIVERDZ'))
'''

In [ ]:
'''
df_new['pgc41'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='CATAST'))
df_new['pgc42'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='CANCRM'))
df_new['pgc43'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='PERINTL'))
df_new['pgc44'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='PNCRDZ'))
df_new['pgc45'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='RENAL1'))
df_new['pgc46'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='SEPSIS'))
'''

In [ ]:
'''
df_new['sp1'] = df_group['Specialty'].transform(lambda x: sum(x=='Internal'))
df_new['sp2'] = df_group['Specialty'].transform(lambda x: sum(x=='Laboratory'))
df_new['sp3'] = df_group['Specialty'].transform(lambda x: sum(x=='General Practice'))
df_new['sp4'] = df_group['Specialty'].transform(lambda x: sum(x=='Surgery'))
df_new['sp5'] = df_group['Specialty'].transform(lambda x: sum(x=='Diagnostic Imaging'))
df_new['sp6'] = df_group['Specialty'].transform(lambda x: sum(x=='Emergency'))
df_new['sp7'] = df_group['Specialty'].transform(lambda x: sum(x=='Other'))
df_new['sp8'] = df_group['Specialty'].transform(lambda x: sum(x=='Pediatrics'))
df_new['sp9'] = df_group['Specialty'].transform(lambda x: sum(x=='Rehabilitation'))
df_new['sp10'] = df_group['Specialty'].transform(lambda x: sum(x=='Obstetrics and Gynecology'))
df_new['sp11'] = df_group['Specialty'].transform(lambda x: sum(x=='Anesthesiology'))
df_new['sp12'] = df_group['Specialty'].transform(lambda x: sum(x=='Pathology'))
df_new['sp13'] = df_group['Specialty'].transform(lambda x: sum(x.isnull()))
'''

In [ ]:
'''
df_new['pg1'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='EM'))
df_new['pg2'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='PL'))
df_new['pg3'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='MED'))                                                                 
df_new['pg4'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SCS'))
df_new['pg5'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='RAD'))
df_new['pg6'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SDS'))
df_new['pg7'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SIS'))
df_new['pg8'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SMS'))                                                                 
df_new['pg9'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='ANES'))
df_new['pg10'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SGS'))
df_new['pg11'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SEOA'))
df_new['pg12'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SRS'))
df_new['pg13'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SNS'))                                                                 
df_new['pg14'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SAS'))
df_new['pg15'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SUS'))
df_new['pg16'] = df_group['ProcedureGroup'].transform(lambda x: sum(x.isnull()))
df_new['pg17'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SMCD'))
df_new['pg18'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SO'))
'''

In [ ]:
'''
df_new['ps1'] = df_group['PlaceSvc'].transform(lambda x: sum(x=='Office'))                   
df_new['ps2'] = df_group['PlaceSvc'].transform(lambda x: sum(x=='Independent Lab'))                                             
df_new['ps3'] = df_group['PlaceSvc'].transform(lambda x: sum(x=='Urgent Care'))                                                             
df_new['ps4'] = df_group['PlaceSvc'].transform(lambda x: sum(x=='Outpatient Hospital'))                                                                
df_new['ps5'] = df_group['PlaceSvc'].transform(lambda x: sum(x=='Inpatient Hospital'))                                                                 
df_new['ps6'] = df_group['PlaceSvc'].transform(lambda x: sum(x=='Ambulance'))                                                                 
df_new['ps7'] = df_group['PlaceSvc'].transform(lambda x: sum(x=='Other'))                                                                 
df_new['ps8'] = df_group['PlaceSvc'].transform(lambda x: sum(x=='Home'))                                                                 
df_new['ps9'] = df_group['PlaceSvc'].transform(lambda x: sum(x.isnull()))
'''  

In [ ]:
df_new.head()

In [40]:
count_missing(df_new)

                                         Total  Percent
CharlsonIndex_std                        18443  0.08444
PayDelay_stdev                           18443  0.08444
DSFS_std                                 18443  0.08444
LOS_std                                  18443  0.08444
Specialty_Count_Internal                     0  0.00000
Specialty_Count_Diagnostic Imaging           0  0.00000
Specialty_Count_Rehabilitation               0  0.00000
Specialty_Count_Pediatrics                   0  0.00000
Specialty_Count_Laboratory                   0  0.00000
ProcedureGroup_Count_SMCD                    0  0.00000
Specialty_Count_Anesthesiology               0  0.00000
CharlsonIndex_range                          0  0.00000
DSFS_range                                   0  0.00000
no_PrimaryConditionGroup_PlaceSvc            0  0.00000
no_ProcedureGroup_PrimaryConditionGroup      0  0.00000
no_ProcedureGroup_Specialty                  0  0.00000
Specialty_Count_Surgery                      0  

,Total,Percent
CharlsonIndex_std,18443,0.08444
PayDelay_stdev,18443,0.08444
DSFS_std,18443,0.08444
LOS_std,18443,0.08444
Specialty_Count_Internal,0,0.00000
...,...,...
PrimaryConditionGroup_Count_METAB1,0,0.00000
PrimaryConditionGroup_Count_HEART4,0,0.00000
PrimaryConditionGroup_Count_ODaBNCA,0,0.00000
PrimaryConditionGroup_Count_HEMTOL,0,0.00000


In [41]:
df_new['PayDelay_stdev'].fillna(-1, inplace=True)
df_new['LOS_std'].fillna(-1, inplace=True)
df_new['DSFS_std'].fillna(-1, inplace=True)
df_new['CharlsonIndex_std'].fillna(-1, inplace=True)

In [42]:
count_missing(df_new)

                                           Total  Percent
ProcedureGroup_Count_SMCD                      0      0.0
Specialty_Count_Anesthesiology                 0      0.0
CharlsonIndex_std                              0      0.0
no_PrimaryConditionGroup_Specialty             0      0.0
no_ProcedureGroup_Specialty                    0      0.0
no_ProcedureGroup_PrimaryConditionGroup        0      0.0
no_PrimaryConditionGroup_PlaceSvc              0      0.0
DSFS_range                                     0      0.0
CharlsonIndex_range                            0      0.0
Specialty_Count_Surgery                        0      0.0
Specialty_Count_Internal                       0      0.0
Specialty_Count_Laboratory                     0      0.0
Specialty_Count_Pediatrics                     0      0.0
Specialty_Count_Rehabilitation                 0      0.0
Specialty_Count_Diagnostic Imaging             0      0.0
Specialty_Count_Emergency                      0      0.0
PrimaryConditi

,Total,Percent
ProcedureGroup_Count_SMCD,0,0.0
Specialty_Count_Anesthesiology,0,0.0
CharlsonIndex_std,0,0.0
no_PrimaryConditionGroup_Specialty,0,0.0
no_ProcedureGroup_Specialty,0,0.0
...,...,...
PrimaryConditionGroup_Count_METAB1,0,0.0
PrimaryConditionGroup_Count_HEART4,0,0.0
PrimaryConditionGroup_Count_ODaBNCA,0,0.0
PrimaryConditionGroup_Count_HEMTOL,0,0.0


In [43]:
df_new.to_csv(path_or_buf='New Data/df_Claims_new.csv', index=False)

In [ ]:
count_missing(df_new)